<a href="https://colab.research.google.com/github/DarkOxygen123/SBU-Hackathon/blob/main/only_imp_Data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Data loading


In [ ]:
import numpy as np
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import gc
warnings.filterwarnings("ignore")

In [ ]:
!pip install imbalanced-learn

In [ ]:
!pip install smote-variants

In [ ]:
from smote_variants import SMOTE

In [ ]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

In [ ]:
import datetime

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold, GroupKFold, GridSearchCV, train_test_split, TimeSeriesSplit
from sklearn import metrics

In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
from scipy.stats import shapiro

In [ ]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [ ]:
# Loading Train Dataset

train_transaction = pd.read_csv('/content/drive/MyDrive/Dataset/train_transaction.csv')
train_identity = pd.read_csv('/content/drive/MyDrive/Dataset/train_identity.csv')

In [ ]:
print(f'train_transaction shape: {train_transaction.shape}')
print(f'train_identity shape: {train_identity.shape}')

train_transaction shape: (590540, 394)
train_identity shape: (144233, 41)


In [ ]:
#mergin the data files for train and testing
merged_train = pd.merge(train_transaction, train_identity, how='left', on='TransactionID')

In [ ]:
merged_train.to_pickle('/content/drive/MyDrive/Dataset/train_merged.pkl')

In [ ]:
del train_transaction
del train_identity

In [ ]:
gc.collect()

8

### Start here

In [ ]:
train_data = pd.read_pickle("/content/drive/MyDrive/Dataset/train_merged.pkl")

In [ ]:
train_data.head()

## Data Analysis

**TransactionID:** This serves as the unique identifier for each transaction and is utilized as the foreign key in the Identity Table.

**isFraud:** A binary indicator (0 or 1) denoting whether a transaction is fraudulent or not.

**TransactionDT:** Represents the timedelta from a specified reference datetime, although it is not an actual timestamp.

**TransactionAMT:** Indicates the payment amount for the transaction, expressed in USD.

**ProductCD:** Refers to the Product Code associated with the transaction. (*Catergorical Feature*)

**card1 - card6:** Encompasses payment card information, including card type, card category, issue bank, country, and other relevant details.

**addr:** Stands for Address.(*addr1, addr2 Catergorical Feature*)

**dist:** Represents the distance.

**P_emaildomain:** Denotes the Purchaser's Email Domain.(*Catergorical Feature*)

**R_emaildomain:** Denotes the Receiver's Email Domain.(*Catergorical Feature*)

**C1-C14:** Pertains to counting, indicating factors such as the number of addresses associated with the payment card. The exact meaning is obfuscated. (*card1-card6 catergorical Feature*)

**D1-D15:** Represent timedelta, indicating the days between the current and previous transactions, among other time-related factors.

**M1-M9:** Refers to match criteria, including aspects like matching names on the card and address.

**Vxxx:** Refers to Vesta engineered rich features, incorporating ranking, counting, and other entity relations.

In [ ]:
# Get information about null/empty values
null_values = train_data.isnull().sum()

# Get unique values and their occurrence rates
unique_values = train_data.nunique()
occurrence_rates = train_data.apply(lambda x: x.value_counts(normalize=True).iloc[0])

# Combine the results into a DataFrame for analysis
analysis_df = pd.DataFrame({
    'Null/Empty Values': null_values,
    'Unique Values': unique_values,
    'Occurrence Rate of Unique Values': occurrence_rates
})

In [ ]:
analysis_df

In [ ]:
del analysis_df
gc.collect()

Reducing Dimentionality based on missing Data

In [ ]:
# Get the list of non-numerical columns
non_numerical_columns = train_data.select_dtypes(exclude=['number']).columns.tolist()
numerical_columns = train_data.select_dtypes(include=['number']).columns.tolist()

In [ ]:
non_numerical_columns

In [ ]:
numerical_columns

## Feature Engineering

In [ ]:
train_data[['P_emaildomain_1', 'P_emaildomain_2', 'P_emaildomain_3']] = train_data['P_emaildomain'].str.split('.', expand=True)
train_data[['R_emaildomain_1', 'R_emaildomain_2', 'R_emaildomain_3']] = train_data['R_emaildomain'].str.split('.', expand=True)

In [ ]:
START_DATE = '2023-01-05'
startdate = datetime.datetime.strptime(START_DATE, "%Y-%m-%d")
train_data["datetime"] = train_data['TransactionDT'].apply(lambda x: (startdate + datetime.timedelta(seconds=x)))

In [ ]:
train_data['browser'] = train_data['id_31'].apply(lambda x: 'chrome' if 'chrome' in str(x).lower() else x)
train_data['browser'] = train_data['browser'].apply(lambda x: 'edge' if 'edge' in str(x).lower() else x)
train_data['browser'] = train_data['browser'].apply(lambda x: 'safari' if 'safari' in str(x).lower() else x)
train_data['browser'] = train_data['browser'].apply(lambda x: 'opera' if 'opera' in str(x).lower() else x)
train_data['browser'] = train_data['browser'].apply(lambda x: 'firefox' if 'firefox' in str(x).lower() else x)
train_data['browser'] = train_data['browser'].apply(lambda x: 'google' if 'google' in str(x).lower() else x)
train_data['browser'] = train_data['browser'].apply(lambda x: 'blackberry' if 'blackberry' in str(x).lower() else x)
train_data['browser'] = train_data['browser'].apply(lambda x: 'samsung' if 'samsung' in str(x).lower() else x)
train_data['browser'] = train_data['browser'].apply(lambda x: 'ie' if 'ie ' in str(x).lower() else x)

In [ ]:
train_data['os'] = train_data['id_30'].apply(lambda x: 'windows' if 'windows' in str(x).lower() else x)
train_data['os'] = train_data['os'].apply(lambda x: 'ios' if 'ios' in str(x).lower() else x)
train_data['os'] = train_data['os'].apply(lambda x: 'mac' if 'mac' in str(x).lower() else x)
train_data['os'] = train_data['os'].apply(lambda x: 'android' if 'android' in str(x).lower() else x)
train_data['os'] = train_data['os'].apply(lambda x: 'linux' if 'linux' in str(x).lower() else x)

Reducing Dimensions

In [ ]:
many_null_cols = [col for col in train_data.columns if train_data[col].isnull().sum() / train_data.shape[0] > 0.9]


In [ ]:
big_top_value_cols = [col for col in train_data.columns if train_data[col].value_counts(dropna=False, normalize=True).values[0] > 0.9]

In [ ]:
cols_to_drop = list(set(many_null_cols + big_top_value_cols))
#cols_to_drop

In [ ]:
cols_to_drop.remove("isFraud")
len(cols_to_drop)

68

In [ ]:
train_new = train_data.drop(cols_to_drop, axis=1)

In [ ]:
train_new.shape

(590540, 375)

In [ ]:
del train_data
gc.collect()

36

In [ ]:
gc.collect()

0

## Setting cols for loading into model


In [ ]:
# Get the list of non-numerical/ numerical columns
non_numerical_columns = train_new.select_dtypes(exclude=['number']).columns.tolist()
numerical_columns = train_new.select_dtypes(include=['number']).columns.tolist()

In [ ]:
print(non_numerical_columns)
print(numerical_columns)
print(len(non_numerical_columns))
print(len(numerical_columns))

['ProductCD', 'card4', 'card6', 'P_emaildomain', 'R_emaildomain', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'id_12', 'id_15', 'id_16', 'id_28', 'id_29', 'id_30', 'id_31', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType', 'DeviceInfo', 'P_emaildomain_1', 'P_emaildomain_2', 'R_emaildomain_1', 'R_emaildomain_2', 'datetime']
['TransactionID', 'isFraud', 'TransactionDT', 'TransactionAmt', 'card1', 'card2', 'card3', 'card5', 'addr1', 'addr2', 'dist1', 'C1', 'C2', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D8', 'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V29', 'V30', 'V31', 'V32', 'V33', 'V34', 'V35', 'V36', 'V37', 'V38', 'V39', 'V40', 'V41', 'V42', 'V43', 'V44', 'V45', 'V46', 'V47', 'V48', 'V49', 'V50', 'V51',

In [ ]:
for col in non_numerical_columns:
    le = LabelEncoder()
    le.fit(list(train_new[col].astype(str).values))
    train_new[col] = le.transform(list(train_new[col].astype(str).values))

In [ ]:
del non_numerical_columns
del numerical_columns

In [ ]:
X_train = train_new.sort_values('datetime').drop(['isFraud', 'TransactionDT', 'TransactionID', 'datetime'], axis=1)
Y_train = train_new.sort_values('datetime')['isFraud']

In [ ]:
def clean_inf_nan(df):
    return df.replace([np.inf, -np.inf], np.nan)

X_train = clean_inf_nan(X_train)

In [ ]:
gc.collect()

0

## Reduce Memory usage by dataFrame

https://www.kaggle.com/code/gemartin/load-data-reduce-memory-usage


In [ ]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:

           if col_type == 'datetime64[ns]':
            continue

            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

In [ ]:
X_train = reduce_mem_usage(X_train)

Memory usage of dataframe is 1667.02 MB
Memory usage after optimization is: 1667.02 MB
Decreased by 0.0%


In [ ]:
del train_new

In [ ]:
gc.collect()

0

### Checking class imbalance

In [ ]:
print(f'Number of Class 0: {len(Y_train)-sum(Y_train)}, Number of Class 1: {sum(Y_train)}')

Number of Class 0: 569877, Number of Class 1: 20663


In [ ]:
gc.collect()

0

SMOTE to correct for class imbalance

In [ ]:
X_train.head()

,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,P_emaildomain,R_emaildomain,C1,C2,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97,V99,V100,V126,V127,V128,V130,V131,V138,V139,V140,V141,V142,V143,V144,V145,V146,V147,V148,V149,V150,V151,V152,V153,V154,V155,V156,V157,V158,V159,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V282,V283,V285,V287,V288,V289,V291,V292,V294,V302,V303,V304,V306,V307,V308,V310,V312,V313,V314,V315,V317,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339,id_01,id_02,id_03,id_04,id_05,id_06,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_19,id_20,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo,P_emaildomain_1,P_emaildomain_2,R_emaildomain_1,R_emaildomain_2
0,68.5,4,13926,NaN,150.0,1,142.0,1,315.0,87.0,19.0,32,32,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,NaN,13.0,NaN,NaN,NaN,NaN,NaN,13.0,13.0,NaN,NaN,NaN,0.0,1,1,1,2,0,1,2,2,2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,0.0,117.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,3,2,NaN,NaN,NaN,2,2,74,103,NaN,260,4,2,2,2,2,2,1742,25,0,25,0
1,29.0,4,2755,404.0,150.0,2,102.0,1,325.0,87.0,NaN,16,32,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,2,2,2,0,1,1,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [ ]:
nan_counts = X_train.isnull().sum()

# Display columns with NaN values and their respective counts
columns_with_nan = nan_counts[nan_counts > 0]
print("Columns with NaN values:")
columns_with_nan

Columns with NaN values:


card2      8933
card3      1565
card5      4259
addr1     65706
addr2     65706
dist1    352271
D1         1269
D2       280797
D3       262878
D4       168922
D5       309841
D6       517353
D8       515614
D9       515614
D10       76022
D11      279287
D12      525823
D13      528588
D14      528353
D15       89113
V1       279287
V2       279287
V3       279287
V4       279287
V5       279287
V6       279287
V7       279287
V8       279287
V9       279287
V10      279287
V11      279287
V12       76073
V13       76073
V14       76073
V15       76073
V16       76073
V17       76073
V18       76073
V19       76073
V20       76073
V21       76073
V22       76073
V23       76073
V24       76073
V25       76073
V26       76073
V27       76073
V28       76073
V29       76073
V30       76073
V31       76073
V32       76073
V33       76073
V34       76073
V35      168969
V36      168969
V37      168969
V38      168969
V39      168969
V40      168969
V41      168969
V42      168969
V43     

### Filling Nan values with Mean

In [ ]:
column_means = X_train.mean()

# Replace NaN values with the mean for each column
data_filled_mean = X_train.apply(lambda col: col.fillna(column_means[col.name]))

In [ ]:
del X_train

In [ ]:
gc.collect()

0

Resampling with SMOTE

In [ ]:
print(data_filled_mean.shape)
print(Y_train.shape)

(590540, 371)
(590540,)


In [ ]:
gc.collect()

0

In [ ]:
data_filled_mean = reduce_mem_usage(data_filled_mean)

Memory usage of dataframe is 1667.02 MB
Memory usage after optimization is: 1667.02 MB
Decreased by 0.0%


In [ ]:
from sklearn.utils import shuffle

In [ ]:
class_1_resample_percentage = 0.5

# Shuffle and select samples for Class 1
class_1_samples = shuffle(data_filled_mean[Y_train == 1], random_state=42).iloc[:int(len(data_filled_mean) * class_1_resample_percentage)]

# Combine the samples for both classes
X_train_balanced = pd.concat([data_filled_mean[Y_train == 0], class_1_samples])
y_train_balanced = pd.concat([Y_train[Y_train == 0], Y_train.loc[class_1_samples.index]])

# Shuffle the combined dataset
X_train_balanced, y_train_balanced = shuffle(X_train_balanced, y_train_balanced, random_state=42)


In [ ]:
data_filled_mean, Y_train_resampled = SMOTE().fit_resample(data_filled_mean, Y_train)

In [ ]:
# Apply Random Over-sampling (ROS)
ros = RandomOverSampler(random_state=42)
data_filled_mean, Y_train = ros.fit_resample(data_filled_mean, Y_train)

# Convert resampled data to PyTorch tensors
data_filled_mean = torch.tensor(data_filled_mean.values, dtype=torch.float32)
Y_train = torch.tensor(Y_train.values, dtype=torch.int64)

# Apply Random Under-sampling (RUS)
rus = RandomUnderSampler(random_state=42)
data_filled_mean, Y_train = rus.fit_resample(data_filled_mean, Y_train)

# Convert resampled data to PyTorch tensors
data_filled_mean = torch.tensor(data_filled_mean.values, dtype=torch.float32)
Y_train = torch.tensor(Y_train.values, dtype=torch.int64)


STORING PREPROCESSED DATA

In [ ]:
type(data_filled_mean)

pandas.core.frame.DataFrame

In [ ]:
print(X_train_balanced.shape)
print(y_train_balanced.shape)
print(len(X_train_balanced))
print(sum(y_train_balanced))

(590540, 371)
(590540,)
590540
20663


In [ ]:
X_train_balanced.to_pickle("/content/drive/MyDrive/Dataset/X_processed")
y_train_balanced.to_pickle("/content/drive/MyDrive/Dataset/Y_processed")